In [435]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from dotenv import load_dotenv

In [394]:

mopdel_1 = True
def model_2_init():
    input_shape=(255,255,3)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32,kernel_size=(5,5),input_shape=input_shape,padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(64,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(4,4)))
    model.add(tf.keras.layers.Conv2D(128,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(160,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))
    model.add(tf.keras.layers.Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(30,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(2,activation='softmax'))

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.Recall()])
    return model


In [395]:
model = model_2_init()


In [396]:
model.summary()

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_230 (Conv2D)         (None, 255, 255, 32)      2432      
                                                                 
 conv2d_231 (Conv2D)         (None, 255, 255, 64)      51264     
                                                                 
 max_pooling2d_92 (MaxPooli  (None, 63, 63, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_232 (Conv2D)         (None, 63, 63, 128)       131200    
                                                                 
 conv2d_233 (Conv2D)         (None, 63, 63, 160)       327840    
                                                                 
 max_pooling2d_93 (MaxPooli  (None, 21, 21, 160)       0         
 ng2D)                                               

In [438]:
dir_train = os.environ.get('TRAIN_PATH')
dir_test = os.environ.get('TEST_PATH')
dir_val = os.environ.get('VAL_PATH')

In [439]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

Found 130 files belonging to 2 classes.


In [441]:
dataset_val = image_dataset_from_directory(
    dir_val,
    labels = "inferred",
    label_mode = "categorical",
    seed=123,
    image_size=(255,255),
    batch_size=8
)

Found 318 files belonging to 2 classes.


In [442]:
dataset_test = image_dataset_from_directory(
    dir_test,
    labels = 'inferred',
    label_mode = 'categorical',
    seed=123,
    image_size=(255,255),
    batch_size=8,
)

Found 130 files belonging to 2 classes.


In [443]:
# datagen
datagen = ImageDataGenerator(
    featurewise_center = False,
    featurewise_std_normalization = False,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = (0.8, 1.2),
)

In [402]:
# dataset_train = datagen.flow(dataset_train)

# Apply data augmentation and prefetch the data
dataset_train = dataset_train.map(lambda x, y: (datagen.random_transform(x), y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

TypeError: in user code:

    File "/var/folders/9q/11j0dpr15c9_qbrj52ywv30r0000gn/T/ipykernel_2317/2655034178.py", line 4, in None  *
        lambda x, y: (datagen.random_transform(x), y)
    File "/Users/gasparburgi/.pyenv/versions/3.10.6/envs/dog_buster/lib/python3.10/site-packages/keras/src/preprocessing/image.py", line 2058, in random_transform  **
        params = self.get_random_transform(x.shape, seed)
    File "/Users/gasparburgi/.pyenv/versions/3.10.6/envs/dog_buster/lib/python3.10/site-packages/keras/src/preprocessing/image.py", line 1925, in get_random_transform
        tx *= img_shape[img_row_axis]

    TypeError: unsupported operand type(s) for *=: 'float' and 'NoneType'


In [370]:
#es_recall = EarlyStoppingByRecall(monitor='val_recall', value=0.90, verbose=1)
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
model = model_2_init()
history = model.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

Epoch 1/20
17/17 [==============================] - 35s 2s/step - loss: 22.7807 - recall_17: 0.5769 - val_loss: 0.3807 - val_recall_17: 0.9057
Epoch 2/20
17/17 [==============================] - 36s 2s/step - loss: 0.6482 - recall_17: 0.6923 - val_loss: 0.3538 - val_recall_17: 0.9057
Epoch 3/20
17/17 [==============================] - 38s 2s/step - loss: 0.7343 - recall_17: 0.7615 - val_loss: 0.3255 - val_recall_17: 0.9057
Epoch 4/20
17/17 [==============================] - 41s 2s/step - loss: 0.5841 - recall_17: 0.7538 - val_loss: 0.4423 - val_recall_17: 0.9057


In [ ]:
def load_img(path):
    img = Image.open(path)
    img = img.resize((255,255))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = np.resize(img,(255,255,3))
    img = img.reshape((-1,255,255,3))
    return img

In [ ]:
imgs = [
    '/Users/gasparburgi/code/lacorreyann/dog_buster/raw_data/Datasets/Dataset_Test/1/IMG_1688.jpg',
    ''
    ]


In [403]:

img = Image.open('/Users/gasparburgi/code/lacorreyann/dog_buster/raw_data/Datasets/Dataset_Test/1/IMG_1688.jpg')

<class 'PIL.JpegImagePlugin.JpegImageFile'>
(4032, 3024, 3)
<class 'numpy.ndarray'>


In [415]:
res = model.predict(img)


1/1 [==============================] - 0s 61ms/step


In [416]:
print(res)

[[0.81655836 0.18344158]]


In [421]:
# Obtener la categoria con más probabilidad
dog_id = np.argmax(res.max())
print(dog_id)

0
